In [11]:
from __future__ import annotations

from pathlib import Path
from typing import Union

from mpi4py import MPI
from petsc4py import PETSc

import dolfinx.fem as fem
import numpy as np
import scipy.sparse.linalg
from dolfinx import default_scalar_type
from dolfinx.common import Timer, TimingType, list_timings
from dolfinx.io import XDMFFile
from dolfinx.mesh import create_unit_square, locate_entities_boundary
from ufl import (SpatialCoordinate, TestFunction, TrialFunction, as_vector, dx,
                 exp, grad, inner, pi, sin)

import dolfinx_mpc.utils
from dolfinx_mpc import LinearProblem, MultiPointConstraint

In [12]:
# Get PETSc int and scalar types
complex_mode = True if np.dtype(default_scalar_type).kind == 'c' else False

In [13]:
NX = 50
NY = 100
mesh = create_unit_square(MPI.COMM_WORLD, NX, NY)
V = fem.functionspace(mesh, ("Lagrange", 1, (mesh.geometry.dim, )))
tol = 250 * np.finfo(default_scalar_type).resolution

In [14]:
def dirichletboundary(x):
    return np.logical_or(np.isclose(x[1], 0, atol=tol), np.isclose(x[1], 1, atol=tol))

In [15]:
facets = locate_entities_boundary(mesh, 1, dirichletboundary)
topological_dofs = fem.locate_dofs_topological(V, 1, facets)
zero = np.array([0, 0], dtype=default_scalar_type)
bc = fem.dirichletbc(zero, topological_dofs, V)
bcs = [bc]

In [16]:
def periodic_boundary(x):
    return np.isclose(x[0], 1, atol=tol)
def periodic_relation(x):
    out_x = np.zeros_like(x)
    out_x[0] = 1 - x[0]
    out_x[1] = x[1]
    out_x[2] = x[2]
    return out_x
with Timer("~PERIODIC: Initialize MPC"):
    mpc = MultiPointConstraint(V)
    mpc.create_periodic_constraint_geometrical(V, periodic_boundary, periodic_relation, bcs)
    mpc.finalize()

In [17]:
u = TrialFunction(V)
v = TestFunction(V)
a = inner(grad(u), grad(v)) * dx

In [19]:
x = SpatialCoordinate(mesh)
dx_ = x[0] - 0.9
dy_ = x[1] - 0.5
f = as_vector((x[0] * sin(5.0 * pi * x[1]) + 1.0 * exp(-(dx_ * dx_ + dy_ * dy_) / 0.02), 0.3 * x[1]))
rhs = inner(f, v) * dx
# Setup MPC system
with Timer("~PERIODIC: Initialize varitional problem"):
    problem = LinearProblem(a, rhs, mpc, bcs=bcs)
solver = problem.solver
solver_prefix = "dolfinx_mpc_solve_{}".format(id(solver))
solver.setOptionsPrefix(solver_prefix)
petsc_options: dict[str, Union[str, int, float]]
if complex_mode or default_scalar_type == np.float32:
    petsc_options = {"ksp_type": "preonly", "pc_type": "lu"}
else:
    petsc_options = {"ksp_type": "cg", "ksp_rtol": 1e-6, "pc_type": "hypre", "pc_hypre_type": "boomeramg",
                     "pc_hypre_boomeramg_max_iter": 1, "pc_hypre_boomeramg_cycle_type": "v"  # ,
                     # "pc_hypre_boomeramg_print_statistics": 1
                     }

In [20]:
# Set PETSc options
opts = PETSc.Options()  # type: ignore
opts.prefixPush(solver_prefix)
if petsc_options is not None:
    for k, v in petsc_options.items():
        opts[k] = v
opts.prefixPop()
solver.setFromOptions()

In [28]:
with Timer("~PERIODIC: Assemble and solve MPC problem"):
    uh = problem.solve()
    # solver.view()
    it = solver.getIterationNumber()
    print("Constrained solver iterations {0:d}".format(it))

Constrained solver iterations 1


# Poisson non-periodic

In [1]:
from mpi4py import MPI
from dolfinx.io import XDMFFile, gmshio
from dolfinx_mpc import LinearProblem, MultiPointConstraint
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot, default_scalar_type
from dolfinx.common import Timer
from ufl import ds, dx, grad, inner
from typing import Union

In [2]:
# Get PETSc int and scalar types
complex_mode = True if np.dtype(default_scalar_type).kind == 'c' else False
print(complex_mode)

True


In [3]:
msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0, 0.0), (2.0, 1.0)),
    n=(64, 32),
    cell_type=mesh.CellType.triangle,
)
V = fem.functionspace(msh, ("Lagrange", 1))

In [4]:
facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.logical_or(np.isclose(x[1], 0.0), np.isclose(x[1], 1.0)),
)
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)
bcs=[]

In [8]:
tol = 250 * np.finfo(default_scalar_type).resolution

In [15]:
def periodic_boundary(x):
    return np.isclose(x[0], 2, atol=tol)
def periodic_relation(x):
    out_x = np.zeros_like(x)
    out_x[0] = 2 - x[0]
    out_x[1] = x[1]
    out_x[2] = x[2]
    return out_x
with Timer("~PERIODIC: Initialize MPC"):
    mpc = MultiPointConstraint(V)
    mpc.create_periodic_constraint_geometrical(V, periodic_boundary, periodic_relation, bcs)
    mpc.finalize()

TypeError: MultiPointConstraint.create_periodic_constraint_geometrical() missing 1 required positional argument: 'bcs'

In [16]:
mpc.create_periodic_constraint_geometrical?

Signature:
mpc.create_periodic_constraint_geometrical(
    V: '_fem.FunctionSpaceBase',
    indicator: 'Callable[[numpy.ndarray], numpy.ndarray]',
    relation: 'Callable[[numpy.ndarray], numpy.ndarray]',
    bcs: 'List[_fem.DirichletBC]',
    scale: '_float_classes' = (1+0j),
)
Docstring:
Create a periodic condition for all degrees of freedom whose physical location satisfies
:math:`indicator(x_i)==True`, i.e.
:math:`u(x_i) = scale * u(relation(x_i))` for all :math:`x_i`

Args:
    V: The function space to assign the condition to. Should either be the space of the MPC or a sub space.
    indicator: Lambda-function to locate degrees of freedom that should be slaves
    relation: Lambda-function describing the geometrical relation to master dofs
    bcs: Dirichlet boundary conditions for the problem
         (Periodic constraints will be ignored for these dofs)
    scale: Float for scaling bc
File:      ~/micromamba/envs/FEniCSx/lib/python3.11/site-packages/dolfinx_mpc/multipointconstra

In [10]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = ufl.sin(np.pi)
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx + inner(g, v) * ds

In [13]:
with Timer("~PERIODIC: Initialize varitional problem"):
    problem = LinearProblem(a, L, mpc, bcs=bcs)
solver = problem.solver
petsc_options: dict[str, Union[str, int, float]]
if complex_mode or default_scalar_type == np.float32:
    petsc_options = {"ksp_type": "preonly", "pc_type": "lu"}
else:
    petsc_options = {"ksp_type": "cg", "ksp_rtol": 1e-6, "pc_type": "hypre", "pc_hypre_type": "boomeramg",
                     "pc_hypre_boomeramg_max_iter": 1, "pc_hypre_boomeramg_cycle_type": "v"  # ,
                     # "pc_hypre_boomeramg_print_statistics": 1
                     }
with Timer("~PERIODIC: Assemble and solve MPC problem"):
    uh = problem.solve()
    # solver.view()
    it = solver.getIterationNumber()
    print("Constrained solver iterations {0:d}".format(it))

Constrained solver iterations 10000


In [ ]:
import pyvista
cells, types, x = plot.vtk_mesh(V)
grid = pyvista.UnstructuredGrid(cells, types, x)
grid.point_data["u"] = uh.x.array.real
grid.set_active_scalars("u")
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
warped = grid.warp_by_scalar()
plotter.add_mesh(warped)
if pyvista.OFF_SCREEN:
    pyvista.start_xvfb(wait=0.1)
    plotter.screenshot("uh_poisson.png")
else:
    plotter.show()

# Liouville

In [1]:
from mpi4py import MPI
from dolfinx.io import XDMFFile, gmshio
from dolfinx_mpc import LinearProblem, MultiPointConstraint
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot, default_scalar_type
from dolfinx.common import Timer
from ufl import ds, dx, grad, inner
from typing import Union

In [9]:
msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0, 0.5), (2.0*np.pi, 1.0)),
    n=(64, 32),
    cell_type=mesh.CellType.triangle,
)
V = fem.functionspace(msh, ("Lagrange", 1))

In [10]:
facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.logical_or(np.isclose(x[1], 0.0), np.isclose(x[1], 1.0)),
)
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)
bcs=[bc]

In [11]:
tol = 250 * np.finfo(default_scalar_type).resolution
def periodic_boundary(x):
    return np.isclose(x[0], 2, atol=tol)
def periodic_relation(x):
    out_x = np.zeros_like(x)
    out_x[0] = 2 - x[0]
    out_x[1] = x[1]
    out_x[2] = x[2]
    return out_x
with Timer("~PERIODIC: Initialize MPC"):
    mpc = MultiPointConstraint(V)
    mpc.create_periodic_constraint_geometrical(V, periodic_boundary, periodic_relation, bcs)
    mpc.finalize()

In [12]:
lmbda = 1.0e-02  # surface parameter
dt = 5.0e-06  # time step
theta = 0.5  # time stepping family, e.g. theta=1 -> backward Euler, theta=0.5 -> Crank-Nicholson

In [16]:
u = fem.Function(V)
v = ufl.TestFunction(V)

In [15]:
x = ufl.SpatialCoordinate(msh)
u0= exp(-(x[1]-0.75)**2/0.1)

In [ ]:

x = ufl.SpatialCoordinate(msh)
f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = ufl.sin(np.pi)
a = inner(grad(u), grad(v)) * dx
L = inner(f, v) * dx + inner(g, v) * ds